In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
y = np.squeeze(np.load("jpca_reduced.npy"))

In [107]:
with np.load("clock_wandering_01.npz") as data:
    x = np.squeeze(data['x'])
    y = np.squeeze(data['y'])
    print(data['y'].shape)

(1, 10000, 2)


In [4]:
y.shape

(6, 2268)

In [11]:
np.savez("jpca_reduced_sc.npz", x=obs, y=y.T[None,:,:])

In [7]:
%matplotlib qt
fig, ax = plt.subplots()
t = 21

cps = np.array([[0.223, -0.301],[0,0]])
cps = np.fliplr(cps)
cps[:,0] *= -1

for i in range(y.shape[1]//t):
    
    # ax.plot(y[0,(i+1)*t-1], y[1,(i+1)*t-1], 'r.')
    if cps[0,:] @ y[:2,i*t] >0:
        c = 'C0'
    else:
        c = 'C1'
    ax.plot(y[0,i*t:t*(i+1)], y[1,i*t:(i+1)*t], color=c)
ax.plot(cps[:,0], cps[:,1])



In [8]:
obs = np.zeros(y.shape[1])
trail_length = 21
for i in range(obs.shape[0]//trail_length):
    b = -1 if np.array([0.301,  0.223]) @ y[:2,i*trail_length] > 0 else 1
    obs[i*trail_length:(i+1)*trail_length] = np.arange(trail_length) * b


In [10]:
plt.plot(obs)

In [126]:
obs.shape

(2268,)

In [63]:
%matplotlib qt
ax = plt.figure().add_subplot(projection='3d')
s = slice(0,500)
ax.plot(data_dict['finger_pos'][s,0],data_dict['finger_pos'][s,1], data_dict['finger_pos'][s,2])


# Pranjal's Code

In [11]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.signal as signal
import mat73
from proSVD import proSVD

In [12]:
data_dict = mat73.loadmat('indy_20160407_02.mat')

units_unsorted = []
units_sorted = []

for ch_curr in data_dict['spikes']:
    units_unsorted.append(ch_curr[0]) # from data description, first unit is unsorted
    for unit_curr in ch_curr[1:]:
        if unit_curr is not None:
            units_sorted.append(unit_curr)

In [24]:
bin_size_ms = 10
timesteps = int(1000 / bin_size_ms) # todo: what is this for?

mintime, maxtime = 100, 0
for spk_times_curr in units_sorted:
    mintime = min(spk_times_curr[0], mintime)
    maxtime = max(spk_times_curr[-1], maxtime)

In [27]:
spk_bins = np.arange(mintime, maxtime, bin_size_ms/1000)
spks_binned = []

for i, unit_curr in enumerate(units_sorted):
    spks_binned_curr, _ = np.histogram(unit_curr, bins=spk_bins, range=(spk_bins[0], spk_bins[-1]))
    spks_binned.append(spks_binned_curr)

data = np.array(spks_binned).T

In [64]:
spk_bins

array([ 60.00056373,  60.01056373,  60.02056373, ..., 887.75056373,
       887.76056373, 887.77056373])

In [40]:
l1 = 100 # columns used to initialize
l = 1 # columns per update
k=6 # reduced dimension

num_iters = np.floor((data.shape[0] - l1 - l)/l).astype('int')
update_times = np.arange(l1, num_iters*l, l) # index of when updates happen (not including init)

In [41]:
kern_sd = 50
smooth_filt = signal.gaussian(int(6 * kern_sd / bin_size_ms), int(kern_sd / bin_size_ms), sym=True)
smooth_filt /=  np.sum(smooth_filt)
data_init_smooth = np.apply_along_axis(lambda x, filt: np.convolve(x, filt, 'same'), 
                                       0, data[:l1, :], filt=smooth_filt)

In [42]:
pro = proSVD(
    k=k,
    decay_alpha=1, 
    trueSVD=True, 
    history=0
)

pro.initialize(data_init_smooth.T)

In [43]:
data_red = np.zeros((data.shape[0], k))

data_red[:l1, :] = data_init_smooth @ pro.Q



pro_end = int(data.shape[0]/8) # when proSVD will stop updating and bubblewrap starts


pro_diffs = []

smooth_window = data[l1-len(smooth_filt):l1, :]

for i, t in enumerate(update_times): 

    start, end = t, t+pro.w_len

    dat_curr = data[start:end, :]

    smooth_window[:-1, :] = smooth_window[1:, :]

    smooth_window[-1, :] = dat_curr

    dat_smooth = smooth_filt @ smooth_window

    

    # proSVD updates

    if t < pro_end:  # only update for first n steps

        pro.preupdate()

        pro.updateSVD(dat_smooth[:, None])

        pro.postupdate()

        pro_diffs.append(np.linalg.norm(pro.Q-pro.Q_prev, axis=0))

        

    # getting projected data

    data_red[start:end, :] = dat_smooth @ pro.Q


In [ ]:
# %%

# Plotting - proSVD changes

fig, ax = plt.subplots()

pro_diffs = np.array(pro_diffs)

ax.plot(np.arange(pro_diffs.shape[0])/timesteps, pro_diffs)

ax.set(title='proSVD stabilizes with <1 min of data', 

       xlabel='seconds of data seen', 

       ylabel=r'$|\Delta\mathbf{Q}|$')

​

# proSVD-reduced trials

fig, ax = plt.subplots(2, 2, figsize=(12, 12), sharex=True, sharey=True)

plt.subplots_adjust(hspace=0, wspace=0)

ax = ax.flatten()

for j, tr_start in enumerate([1, 10, 50, 500]):

    n_trials_plot = 5

    plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,n_trials_plot)))

    for i in range(n_trials_plot):

        start_bin = (tr_start+i) * timesteps

        xs, ys = [*data_red[start_bin:start_bin+timesteps, :2].T]

        ax[j].plot(xs, ys)

​

​

#%%

# bubblewrap learning

time_sec = np.array(bw.pred).shape[0]/(1000/bin_size_ms)

fig, ax = plt.subplots()

axtwin = plot_pred_entropy(bw, ax=ax, bin_size_ms=bin_size_ms)

# axs[1].set(xlim=(0,1000), ylim=(-40, 15))

ax.set(title='log predictive probability (above) and entropy (below)')

ax.set_xlabel('Time (s)')

ax.set(ylim=(-25, 15),

    yticks=np.arange(-10, 20, 5))

axtwin.set(ylim=(0,20),

    yticks=np.arange(0,7,2)) 


#%%

# prediction trial

for trial in np.arange(700, 701):

    start, end = trial*timesteps, (trial+1)*timesteps

    newtrial = data_red[start:end, :]

    fig, ax = plt.subplots(figsize=(5,7))

    ax = draw_future(bw, ax, n_bubbles_forward=8, n_conns=1, trial=newtrial[:-35], n_steps_in=15)

    ax.set(xlim=(-1.1,-.45), ylim=(-.4, .27))

    full_trials = np.floor( data_red[pro_end:int(T/2)].shape[0] / timesteps).astype('int')

    ind = 0

    for j in range(full_trials):

        ax.plot(data_red[ind:ind+timesteps, 0], data_red[ind:ind+timesteps, 1], 

                alpha=.01, color='k', rasterized=True, zorder=99) 

        ind += timesteps

    # if trial == 700:

    #     plt.savefig('bw_predict_trial.svg', bbox_inches='tight')